In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [10]:
# Dataset - https://www.datacamp.com/community/tutorials/amazon-web-scraping-using-beautifulsoup?utm_source=adwords_ppc&utm_campaignid=1455363063&utm_adgroupid=65083631748&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=&utm_creative=332602034364&utm_targetid=aud-299261629574:dsa-429603003980&utm_loc_interest_ms=&utm_loc_physical_ms=20453&gclid=Cj0KCQjwoJX8BRCZARIsAEWBFMI-sceIDbJi_f28LG-W0lRWlUDkxpoq7MjiX4HDqzw2pBnWjEImYN4aAq1QEALw_wcB

In [11]:
no_pages = 2

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)
    alls = []
    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        #print(d)
        name = d.find('span', attrs={'class':'zg-text-center-align'})
        n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find('a', attrs={'class':'a-size-small a-link-child'})
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        users_rated = d.find('a', attrs={'class':'a-size-small a-link-normal'})
        price = d.find('span', attrs={'class':'p13n-sc-price'})

        all1=[]
        if name is not None:
            #print(n[0]['alt'])
            all1.append(n[0]['alt'])
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')

        if rating is not None:
            #print(rating.text)
            all1.append(rating.text)
        else:
            all1.append('-1')

        if users_rated is not None:
            #print(price.text)
            all1.append(users_rated.text)
        else:
            all1.append('0')     

        if price is not None:
            #print(price.text)
            all1.append(price.text)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

In [12]:
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Book Name','Author','Rating','Customers_Rated', 'Price'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')

In [13]:
df = pd.read_csv("amazon_products.csv")

In [14]:
df.head()

,Book Name,Author,Rating,Customers_Rated,Price
0,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5 out of 5 stars,"7,704",₹ 399.00
1,One Arranged Murder,Chetan Bhagat,4.1 out of 5 stars,"1,897",₹ 155.00
2,Think Like a Monk,Jay Shetty,4.8 out of 5 stars,"5,494",₹ 373.00
3,World’s Greatest Books For Personal Growth & W...,Dr. Joseph Murphy et al. Dale,4.5 out of 5 stars,"1,275",₹ 299.00
4,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6 out of 5 stars,"5,103",₹ 359.00


In [15]:
df.shape

(100, 5)

In [16]:
df['Rating'] = df['Rating'].apply(lambda x: x.split()[0])

In [17]:
df['Rating'] = pd.to_numeric(df['Rating'])

In [18]:
df["Price"] = df["Price"].str.replace('₹', '')

In [19]:
df["Price"] = df["Price"].str.replace(',', '')

In [20]:
df['Price'] = df['Price'].apply(lambda x: x.split('.')[0])

In [21]:
df['Price'] = df['Price'].astype(int)

In [22]:
df["Customers_Rated"] = df["Customers_Rated"].str.replace(',', '')

In [23]:
df['Customers_Rated'] = pd.to_numeric(df['Customers_Rated'], errors='ignore')

In [24]:
df.head()

,Book Name,Author,Rating,Customers_Rated,Price
0,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5,7704,399
1,One Arranged Murder,Chetan Bhagat,4.1,1897,155
2,Think Like a Monk,Jay Shetty,4.8,5494,373
3,World’s Greatest Books For Personal Growth & W...,Dr. Joseph Murphy et al. Dale,4.5,1275,299
4,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6,5103,359


In [25]:
df.replace(str(0), np.nan, inplace=True)
df.replace(0, np.nan, inplace=True)

In [26]:
df = df.dropna()

In [27]:
data = df.sort_values(["Price"], axis=0, ascending=False)[:15]

In [28]:
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge
import math
from bokeh.io import curdoc
curdoc().clear()
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.models import Legend
output_notebook()

In [29]:
p = figure(x_range=data.iloc[:,1], plot_width=800, plot_height=550, title="Authors Highest Priced Book", toolbar_location=None, tools="")

p.vbar(x=data.iloc[:,1], top=data.iloc[:,4], width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = math.pi/2

In [30]:
show(p)

ERROR:bokeh.core.validation.check:E-1019 (DUPLICATE_FACTORS): FactorRange must specify a unique list of categorical factors for an axis: duplicate factors found: 'M. Laxmikanth'
